<a href="https://colab.research.google.com/github/JoaoMMS98/ML_2.0/blob/main/Final_Model_Selection_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Model Selection

In this notebook, we are going to apply different ways to split your dataset and try to select the best model.

__`Step 1`__ Import the needed libraries

In [1]:
!pip install xgboost==1.7.5
!pip install scikit-learn==1.2.2
!pip install catboost==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 26.4 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Perceptron
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

Chiel > XGBoost,CatBoost.

Mariana > KNN, RFM, SVM

In [3]:
# Import the data
train_df = pd.read_csv('traindf (2).csv')
val_df = pd.read_csv('valdf (2).csv')
test_df = pd.read_csv('testdf (2).csv')

In [4]:
train_df

,Claim Injury Type,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,4. TEMPORARY,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.274579,1.046863,...,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,3. MED ONLY,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,0.384500,1.046863,...,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,2. NON-COMP,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.040655,1.046863,...,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,2. NON-COMP,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.040655,1.046863,...,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,3. MED ONLY,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,0.574898,1.046863,...,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,4. TEMPORARY,-0.289586,1.366290,1.245247,-0.256161,-1.495213,-0.411805,0.983139,-0.275417,1.046863,...,0,0,1,-1.723148,0.040346,-0.501344,0,0,0,0
459216,4. TEMPORARY,-0.048312,1.291941,-0.858729,-0.048463,-1.228493,-0.617431,0.983139,1.274579,1.046863,...,0,0,1,0.831661,-1.053625,0.002000,0,0,0,0
459217,4. TEMPORARY,-0.534036,1.232673,-1.003831,-0.466591,0.558209,-0.618653,0.983139,-0.035105,-1.041200,...,0,0,0,1.045677,1.490061,-0.052608,0,0,0,0
459218,2. NON-COMP,1.069171,-0.755254,1.172696,0.913504,0.558209,-0.322674,0.983139,1.040655,1.046863,...,0,0,1,0.964689,-0.990343,1.342272,0,0,0,0


In [5]:
val_df

,Claim Injury Type,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,...,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,2. NON-COMP,0.177090,-0.755254,-0.641076,0.145570,-0.984268,2.021605,-0.895805,1.040655,1.046863,...,0,0,0,0.881525,-0.863542,0.066105,0,0,0,0
1,2. NON-COMP,-1.143571,-0.755254,1.172696,-0.991300,0.558209,2.021605,-0.895805,1.040655,1.046863,...,0,0,0,-1.483183,-0.323996,-0.090596,0,0,0,0
2,4. TEMPORARY,-1.207065,1.148735,1.100146,-1.045958,0.558209,-0.511726,0.983139,-0.677350,-0.667644,...,0,0,0,1.176614,-0.763331,-0.250859,0,0,0,0
3,2. NON-COMP,-0.797533,-0.755254,-0.568525,-0.693419,0.558209,-0.473659,0.983139,-1.069849,1.046863,...,0,0,1,0.031537,-0.031308,-0.520338,0,0,0,0
4,4. TEMPORARY,1.326319,1.152468,-0.858729,1.134866,0.558209,-0.623720,-0.780636,-0.853001,-0.667644,...,0,0,1,1.040221,1.485711,-0.928712,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114800,4. TEMPORARY,0.173915,1.273293,0.229535,0.142837,0.558209,-0.518970,-0.780636,-1.416376,-1.261856,...,0,0,1,-1.601871,0.073562,-0.913279,0,0,0,0
114801,2. NON-COMP,0.085024,-0.755254,0.374636,0.060851,-1.026043,2.021605,-0.895805,1.274579,1.046863,...,0,0,0,-0.223769,1.753521,3.045807,0,0,0,0
114802,3. MED ONLY,-0.876899,-0.755254,-0.133220,-0.761740,0.558209,-0.622081,-0.780636,-0.959634,-0.667644,...,0,0,1,0.584317,1.572367,-0.040737,0,0,0,0
114803,4. TEMPORARY,1.666008,1.303004,-1.511687,1.421816,0.580704,-0.577546,0.983139,0.574898,1.046863,...,0,0,1,0.125125,0.021484,0.774823,0,0,0,0


In [6]:
train_df = train_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')
val_df = val_df.drop(['Agreement Reached', 'Alternative Dispute Resolution_U', 'Zip Code'], axis=1, errors='ignore')

In [7]:
# Get the column names of both DataFrames
train_cols = set(train_df.columns)
val_cols = set(val_df.columns)

# Find the differences
diff_train = train_cols - val_cols  # Columns in train_df but not in val_df
diff_val = val_cols - train_cols  # Columns in val_df but not in train_df

# Print the differences
if diff_train:
    print("Columns in train_df but not in val_df:")
    print(diff_train)

if diff_val:
    print("\nColumns in val_df but not in train_df:")
    print(diff_val)

if not diff_train and not diff_val:
    print("Both DataFrames have the same columns.")

Both DataFrames have the same columns.


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

test_df['Accident Year'] = scaler.fit_transform(test_df[['Accident Year']])
test_df['First Hearing Date'] = scaler.fit_transform(test_df[['First Hearing Date']])
val_df['Accident Year'] = scaler.fit_transform(val_df[['Accident Year']])
val_df['First Hearing Date'] = scaler.fit_transform(val_df[['First Hearing Date']])
train_df['Accident Year'] = scaler.fit_transform(train_df[['Accident Year']])
train_df['First Hearing Date'] = scaler.fit_transform(train_df[['First Hearing Date']])

In [9]:
categorical_cols = train_df.select_dtypes(include=['object']).columns
num_categorical_cols = len(categorical_cols)
print(f"Number of categorical columns: {num_categorical_cols}")
print(f"Categorical columns: {categorical_cols}")

Number of categorical columns: 1
Categorical columns: Index(['Claim Injury Type'], dtype='object')


## 1.1. The train-test split

8:2 splitting target column

In [10]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Prepare training data
X_train = train_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_train = train_df['Claim Injury Type']  # Target variable

# Prepare validation/test data
X_test = val_df.drop('Claim Injury Type', axis=1)  # Independent variables
y_test = val_df['Claim Injury Type']  # Target variable

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

train:80.0% | test:20.0%

Data shapes:
X_train shape: (459220, 42)
X_test shape: (114805, 42)
y_train shape: (459220,)
y_test shape: (114805,)


In [11]:
X_train

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Accident Month,Accident Day,Days_to_Assembly,Under_20,Age_21_40,Age_41_65,Above_65,Accident Month Sin,Accident Month Cos,Accident Day Sin,Accident Day Cos,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.274579,1.046863,-0.037533,0.920180,-1.101128,0.703471,-0.735036,1.618148,1.767471,0.092321,0.434451,-1.195837,1,0,0,1,0,-1.204709,-0.723357,1.172579,0.785982,1,1,0,1,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,0.384500,1.046863,-1.477658,-0.445038,1.514005,0.185530,-1.267597,1.018289,-0.937642,0.632632,-0.719624,-1.195837,1,0,1,0,0,1.233416,-0.723357,1.172579,0.785982,1,0,1,0,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.040655,1.046863,-0.037533,-0.445038,-0.933566,-1.277371,1.332596,-0.407201,1.259186,-0.447990,0.722970,-1.651072,1,0,0,1,0,-1.393298,-0.012791,0.261973,1.427131,1,0,0,0,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.040655,1.046863,-0.037533,-0.445038,-0.260367,-0.701976,0.572261,-1.077143,-0.175213,0.092321,-0.142587,-0.626794,1,0,1,0,0,0.014354,-1.433924,1.241732,-0.593402,1,0,0,0,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,0.574898,1.046863,-1.413414,-0.445038,-0.417166,-0.256659,-0.390479,1.018289,-0.545659,0.632632,-1.585181,1.194143,1,0,0,1,0,0.718179,1.217947,-1.214956,0.785982,1,1,1,1,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-0.289586,1.366290,1.245247,-0.256161,-1.495213,-0.411805,0.983139,-0.275417,1.046863,0.396110,0.237571,1.514005,-1.189578,-1.322806,-1.060852,1.767471,0.092321,1.011489,-0.057752,1,0,1,0,0,-1.204709,0.697776,0.121123,-1.382412,1,1,1,1,0,0,1,-1.723148,0.040346,-0.501344,0,0,0,0
459216,-0.048312,1.291941,-0.858729,-0.048463,-1.228493,-0.617431,0.983139,1.274579,1.046863,0.171258,3.650616,1.514005,1.290243,0.572261,-0.166136,-0.175213,0.092321,0.145932,1.421760,1,0,0,1,0,-0.689472,-1.243528,-0.824778,1.201208,1,1,1,1,0,0,1,0.831661,-1.053625,0.002000,0,0,0,0
459217,-0.534036,1.232673,-1.003831,-0.466591,0.558209,-0.618653,0.983139,-0.035105,-1.041200,-0.037533,-0.445038,-1.022713,-0.619967,1.332596,1.394296,-0.481046,0.092321,1.300008,-0.512986,1,0,0,1,0,-0.689472,1.217947,1.091177,-0.837745,1,0,0,0,0,0,0,1.045677,1.490061,-0.052608,0,0,0,0
459218,1.069171,-0.755254,1.172696,0.913504,0.558209,-0.322674,0.983139,1.040655,1.046863,-0.037533,-0.445038,-0.579708,0.893373,1.332596,-0.407201,-0.175213,-0.447990,0.145932,1.649377,1,0,1,0,0,-0.689472,-1.243528,-0.304350,1.427131,1,0,0,0,0,0,1,0.964689,-0.990343,1.342272,0,0,0,0


Stratified Hold-Out is used to test the models

# catboost test to ascertain effectiveness of encoded categoricals

In [12]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Identify categorical columns
categorical_features_indices = [i for i, dtype in enumerate(X_train.dtypes)
                              if dtype == 'object' or dtype == 'category']

# Create CatBoost Pools with categorical features
train_pool = Pool(data=X_train,
                 label=y_train,
                 cat_features=categorical_features_indices)

test_pool = Pool(data=X_test,
                label=y_test,
                cat_features=categorical_features_indices)

# Initialize model
model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    verbose=0
)

# Fit using pools
model.fit(train_pool)

# Make predictions for both train and test sets
y_pred_train = model.predict(train_pool)
y_pred_test = model.predict(test_pool)

# Calculate metrics for training set
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

# Calculate metrics for test set
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print(f'\nNumber of categorical features detected: {len(categorical_features_indices)}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

# Calculate performance difference between train and test
print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7860
Macro F1 Score: 0.3858

Testing Metrics:
Accuracy Score: 0.7854
Macro F1 Score: 0.3786

Number of categorical features detected: 0

Detailed Classification Report for Test Set:
              precision    recall  f1-score   support

           0       0.76      0.46      0.57      2495
           1       0.84      0.98      0.91     58216
           2       0.49      0.05      0.09     13781
           3       0.71      0.88      0.78     29701
           4       0.71      0.51      0.60      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       0.57      0.04      0.08        94

    accuracy                           0.79    114805
   macro avg       0.51      0.37      0.38    114805
weighted avg       0.75      0.79      0.74    114805


Train-Test Performance Differences:
Accuracy Difference (Train - Test): 0.0005
F1 Score Difference (Train - Test): 0.0072


# In earlier versions without the proper encoding we seperated categoricals from numericals here.

In [13]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

# Save target variable before dropping categoricals
y_train = train_df['Claim Injury Type']  # Target variable
y_test = val_df['Claim Injury Type']  # Target variable

# Function to remove categorical columns
def drop_categorical_columns(df):
    categorical_cols = df.select_dtypes(include=['object', 'category']).columns
    return df.drop(columns=categorical_cols)

# Drop categorical columns from train and validation sets
X_train = drop_categorical_columns(train_df)
X_test = drop_categorical_columns(val_df)

# Label encode the target variable
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

# Print the split ratio
total_samples = len(train_df) + len(val_df)
print('train:{}% | test:{}%'.format(
    round(len(train_df)/total_samples * 100, 2),
    round(len(val_df)/total_samples * 100, 2)
))

# Optional: Display all columns
pd.set_option('display.max_columns', None)

# Verify shapes and remaining features
print("\nData shapes:")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Print information about removed features
original_features = train_df.shape[1] - 1  # Subtract 1 for target column
remaining_features = X_train.shape[1]
print(f"\nFeature reduction:")
print(f"Original number of features: {original_features}")
print(f"Remaining numeric features: {remaining_features}")
print(f"Number of categorical features removed: {original_features - remaining_features}")

train:80.0% | test:20.0%

Data shapes:
X_train shape: (459220, 42)
X_test shape: (114805, 42)
y_train shape: (459220,)
y_test shape: (114805,)

Feature reduction:
Original number of features: 42
Remaining numeric features: 42
Number of categorical features removed: 0


In [14]:
X_train

,Assembly Date,Average Weekly Wage,Birth Year,C-2 Date,C-3 Date,Carrier Name,Carrier Type,County of Injury,District Name,First Hearing Date,IME-4 Count,Industry Code,WCIO Cause of Injury Code,WCIO Nature of Injury Code,WCIO Part Of Body Code,Number of Dependents,Accident Year,Accident Month,Accident Day,Days_to_Assembly,Under_20,Age_21_40,Age_41_65,Above_65,Accident Month Sin,Accident Month Cos,Accident Day Sin,Accident Day Cos,Received_C2,Received_C3,Hearing_held,Attorney/Representative_Y,COVID-19 Indicator_Y,Alternative Dispute Resolution_Y,Gender_M,wcio_pca1,wcio_pca2,region_cluster,high_risk_nature,high_risk_body,high_risk_cause,market_indicator
0,-0.067360,1.213328,-0.495974,-0.064860,-1.263841,-0.575548,0.983139,1.274579,1.046863,-0.037533,0.920180,-1.101128,0.703471,-0.735036,1.618148,1.767471,0.092321,0.434451,-1.195837,1,0,0,1,0,-1.204709,-0.723357,1.172579,0.785982,1,1,0,1,0,0,0,0.610477,0.806579,0.162263,1,0,0,1
1,-0.908646,1.344678,0.955044,-0.783603,0.558209,-0.494377,-0.780636,0.384500,1.046863,-1.477658,-0.445038,1.514005,0.185530,-1.267597,1.018289,-0.937642,0.632632,-0.719624,-1.195837,1,0,1,0,0,1.233416,-0.723357,1.172579,0.785982,1,0,1,0,0,0,0,-0.175047,0.713979,0.277415,0,0,1,1
2,1.012027,-0.755254,-0.133220,0.864313,0.558209,-0.209935,0.983139,1.040655,1.046863,-0.037533,-0.445038,-0.933566,-1.277371,1.332596,-0.407201,1.259186,-0.447990,0.722970,-1.651072,1,0,0,1,0,-1.393298,-0.012791,0.261973,1.427131,1,0,0,0,0,0,1,-0.025755,0.530618,1.125027,0,0,0,0
3,0.056452,-0.755254,1.317798,0.041721,0.558209,-0.625062,-1.524399,1.040655,1.046863,-0.037533,-0.445038,-0.260367,-0.701976,0.572261,-1.077143,-0.175213,0.092321,-0.142587,-0.626794,1,0,1,0,0,0.014354,-1.433924,1.241732,-0.593402,1,0,0,0,0,0,1,-0.467459,-0.375868,-0.223555,0,0,0,0
4,-0.788009,0.889468,0.156984,-0.698884,-1.896900,-0.453747,-1.524399,0.574898,1.046863,-1.413414,-0.445038,-0.417166,-0.256659,-0.390479,1.018289,-0.545659,0.632632,-1.585181,1.194143,1,0,0,1,0,0.718179,1.217947,-1.214956,0.785982,1,1,1,1,0,0,1,0.104639,0.995235,0.774823,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
459215,-0.289586,1.366290,1.245247,-0.256161,-1.495213,-0.411805,0.983139,-0.275417,1.046863,0.396110,0.237571,1.514005,-1.189578,-1.322806,-1.060852,1.767471,0.092321,1.011489,-0.057752,1,0,1,0,0,-1.204709,0.697776,0.121123,-1.382412,1,1,1,1,0,0,1,-1.723148,0.040346,-0.501344,0,0,0,0
459216,-0.048312,1.291941,-0.858729,-0.048463,-1.228493,-0.617431,0.983139,1.274579,1.046863,0.171258,3.650616,1.514005,1.290243,0.572261,-0.166136,-0.175213,0.092321,0.145932,1.421760,1,0,0,1,0,-0.689472,-1.243528,-0.824778,1.201208,1,1,1,1,0,0,1,0.831661,-1.053625,0.002000,0,0,0,0
459217,-0.534036,1.232673,-1.003831,-0.466591,0.558209,-0.618653,0.983139,-0.035105,-1.041200,-0.037533,-0.445038,-1.022713,-0.619967,1.332596,1.394296,-0.481046,0.092321,1.300008,-0.512986,1,0,0,1,0,-0.689472,1.217947,1.091177,-0.837745,1,0,0,0,0,0,0,1.045677,1.490061,-0.052608,0,0,0,0
459218,1.069171,-0.755254,1.172696,0.913504,0.558209,-0.322674,0.983139,1.040655,1.046863,-0.037533,-0.445038,-0.579708,0.893373,1.332596,-0.407201,-0.175213,-0.447990,0.145932,1.649377,1,0,1,0,0,-0.689472,-1.243528,-0.304350,1.427131,1,0,0,0,0,0,1,0.964689,-0.990343,1.342272,0,0,0,0


# Assesment of XGBoost

In [15]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, classification_report

# Convert to XGBoost's DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Enhanced parameters for multiclass classification
params = {
    'objective': 'multi:softmax',          # Multi-class classification
    'num_class': len(np.unique(y_train)),  # Number of classes in the target
    'learning_rate': 0.05,                 # Reduced learning rate for better convergence
    'max_depth': 6,                        # Reduced depth to prevent overfitting
    'min_child_weight': 3,                 # Increased to make splits more conservative
    'subsample': 0.8,                      # Slightly reduced to improve generalization
    'colsample_bytree': 0.8,               # Slightly reduced for better regularization
    'gamma': 0.2,                          # Minimum loss reduction required for a split
    'lambda': 2,                           # L2 regularization to penalize large weights
    'alpha': 1,                            # L1 regularization to encourage sparsity
    'eval_metric': ['mlogloss', 'merror']  # Evaluation metrics
}

model = xgb.train(
    params,
    dtrain,
    num_boost_round=1000,                     # Increased rounds for slower, detailed learning
    evals=[(dtrain, 'train'), (dtest, 'validation')],
    early_stopping_rounds=30,                # Increased for more stability during training
    verbose_eval=50                          # Less frequent logs for cleaner output
)

# Rest of the code remains the same
y_pred_train = model.predict(dtrain)
y_pred_test = model.predict(dtest)

train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')

test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nDetailed Classification Report for Test Set:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Performance Differences:")
print(f'Accuracy Difference (Train - Test): {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Difference (Train - Test): {(train_f1 - test_f1):.4f}')

importance = model.get_score(importance_type='gain')
importance_df = pd.DataFrame.from_dict(importance, orient='index', columns=['importance'])
importance_df = importance_df.sort_values('importance', ascending=False).head(10)

print("\nTop 10 Most Important Features:")
print(importance_df)

[0]	train-mlogloss:1.96044	train-merror:0.23087	validation-mlogloss:1.96056	validation-merror:0.23163
[50]	train-mlogloss:0.73300	train-merror:0.21384	validation-mlogloss:0.73548	validation-merror:0.21515
[100]	train-mlogloss:0.61349	train-merror:0.21015	validation-mlogloss:0.61880	validation-merror:0.21169
[150]	train-mlogloss:0.58127	train-merror:0.20715	validation-mlogloss:0.58982	validation-merror:0.20944
[200]	train-mlogloss:0.56746	train-merror:0.20517	validation-mlogloss:0.57935	validation-merror:0.20839
[250]	train-mlogloss:0.55888	train-merror:0.20338	validation-mlogloss:0.57414	validation-merror:0.20744
[300]	train-mlogloss:0.55222	train-merror:0.20194	validation-mlogloss:0.57090	validation-merror:0.20692
[350]	train-mlogloss:0.54605	train-merror:0.20034	validation-mlogloss:0.56847	validation-merror:0.20639
[400]	train-mlogloss:0.54039	train-merror:0.19870	validation-mlogloss:0.56660	validation-merror:0.20611
[450]	train-mlogloss:0.53523	train-merror:0.19735	validation-mloglo

# Assesment of Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.impute import SimpleImputer

# Initialize median imputer
imputer = SimpleImputer(strategy='median')

# Fit and transform training data
X_train_imputed = imputer.fit_transform(X_train)

# Transform test data using training set statistics
X_test_imputed = imputer.transform(X_test)

# Initialize Logistic Regression classifier
model = LogisticRegression(
    random_state=42,
    max_iter=1000,
    multi_class='auto'
)

# Train model on imputed data
model.fit(X_train_imputed, y_train)

# Make predictions using imputed data
y_pred_train = model.predict(X_train_imputed)
y_pred_test = model.predict(X_test_imputed)

# Calculate metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Delta:")
print(f'Accuracy Δ: {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Δ: {(train_f1 - test_f1):.4f}')

Training Metrics:
Accuracy Score: 0.7630
Macro F1 Score: 0.3881

Testing Metrics:
Accuracy Score: 0.7623
Macro F1 Score: 0.3771

Classification Report:
              precision    recall  f1-score   support

           0       0.68      0.43      0.53      2495
           1       0.84      0.98      0.90     58216
           2       0.32      0.05      0.09     13781
           3       0.68      0.87      0.76     29701
           4       0.63      0.32      0.42      9656
           5       0.00      0.00      0.00       842
           6       0.00      0.00      0.00        20
           7       0.48      0.23      0.31        94

    accuracy                           0.76    114805
   macro avg       0.45      0.36      0.38    114805
weighted avg       0.71      0.76      0.71    114805


Train-Test Delta:
Accuracy Δ: 0.0008
F1 Score Δ: 0.0109


# Assesment of KNNClassifier

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.impute import SimpleImputer
import numpy as np

# Initialize imputer
imputer = SimpleImputer(strategy='median')

# Impute missing values
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Initialize KNN classifier
model = KNeighborsClassifier(
    n_neighbors=5,
    weights='uniform',
    metric='minkowski',
    p=2
)

# Train model on imputed data
model.fit(X_train_imputed, y_train)

# Generate predictions
y_pred_train = model.predict(X_train_imputed)
y_pred_test = model.predict(X_test_imputed)

# Compute metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy Score: {train_accuracy:.4f}')
print(f'Macro F1 Score: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy Score: {test_accuracy:.4f}')
print(f'Macro F1 Score: {test_f1:.4f}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Delta:")
print(f'Accuracy Δ: {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Score Δ: {(train_f1 - test_f1):.4f}')

print("\nModel Parameters:")
print(f"n_neighbors: {model.n_neighbors}")
print(f"metric: {model.metric}")
print(f"classes: {len(np.unique(y_test))}")

Training Metrics:
Accuracy Score: 0.7963
Macro F1 Score: 0.4243

Testing Metrics:
Accuracy Score: 0.7383
Macro F1 Score: 0.3493

Classification Report:
              precision    recall  f1-score   support

           0       0.61      0.39      0.47      2495
           1       0.82      0.96      0.89     58216
           2       0.25      0.09      0.13     13781
           3       0.69      0.77      0.72     29701
           4       0.57      0.39      0.46      9656
           5       0.14      0.01      0.02       842
           6       0.00      0.00      0.00        20
           7       0.71      0.05      0.10        94

    accuracy                           0.74    114805
   macro avg       0.47      0.33      0.35    114805
weighted avg       0.69      0.74      0.70    114805


Train-Test Delta:
Accuracy Δ: 0.0579
F1 Score Δ: 0.0750

Model Parameters:
n_neighbors: 5
metric: minkowski
classes: 8


# Assesment of Random Forest

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.impute import SimpleImputer
import pandas as pd

# Initialize imputer
imputer = SimpleImputer(strategy='median')

# Impute missing values
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# Preserve column names after imputation
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

# Initialize Random Forest
model = RandomForestClassifier(
   n_estimators=1000,
   max_depth=30,
   min_samples_split=10,
   min_samples_leaf=5,
   max_features='sqrt',
   random_state=42,
   n_jobs=-1,
   bootstrap=True,
   oob_score=True,
   class_weight='balanced'
)

# Train on imputed data
model.fit(X_train_imputed, y_train)

# Generate predictions
y_pred_train = model.predict(X_train_imputed)
y_pred_test = model.predict(X_test_imputed)

# Performance metrics
train_accuracy = accuracy_score(y_train, y_pred_train)
train_f1 = f1_score(y_train, y_pred_train, average='macro')
test_accuracy = accuracy_score(y_test, y_pred_test)
test_f1 = f1_score(y_test, y_pred_test, average='macro')

print("Training Metrics:")
print(f'Accuracy: {train_accuracy:.4f}')
print(f'F1: {train_f1:.4f}')

print("\nTesting Metrics:")
print(f'Accuracy: {test_accuracy:.4f}')
print(f'F1: {test_f1:.4f}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred_test))

print("\nTrain-Test Delta:")
print(f'Accuracy Δ: {(train_accuracy - test_accuracy):.4f}')
print(f'F1 Δ: {(train_f1 - test_f1):.4f}')

# Feature importance analysis
importance_df = pd.DataFrame({
   'feature': X_train_imputed.columns,
   'importance': model.feature_importances_
}).nlargest(10, 'importance')

print("\nTop Features:")
print(importance_df)

print("\nModel Parameters:")
print(f"Trees: {model.n_estimators}")
print(f"Depth: {model.max_depth}")
print(f"Features: {model.n_features_in_}")

Training Metrics:
Accuracy: 0.9191
F1: 0.8928

Testing Metrics:
Accuracy: 0.7799
F1: 0.4543

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.62      0.61      2495
           1       0.87      0.95      0.90     58216
           2       0.40      0.15      0.22     13781
           3       0.75      0.81      0.78     29701
           4       0.60      0.70      0.65      9656
           5       0.20      0.02      0.04       842
           6       0.00      0.00      0.00        20
           7       0.36      0.52      0.43        94

    accuracy                           0.78    114805
   macro avg       0.47      0.47      0.45    114805
weighted avg       0.75      0.78      0.75    114805


Train-Test Delta:
Accuracy Δ: 0.1392
F1 Δ: 0.4386

Top Features:
                       feature  importance
1          Average Weekly Wage    0.105020
10                 IME-4 Count    0.073637
35                   wcio_pca1    0.04